In [137]:
import pandas as pd
import numpy as np 
from prophet import Prophet
from statsmodels.tsa.arima.model import ARIMA
from keras.models import Sequential
from keras.layers import Dense, LSTM

df = pd.read_csv('./data/dataset.csv')

# Split into train and test
train = df[df['count'] != '?']
test = df[df['count'] == '?']


In [138]:

train['count'] = pd.to_numeric(train['count'], errors='coerce')
train = train.dropna(subset=['count'])

train['count'] = train['count'].astype(int)


model_arima = ARIMA(train['count'], order=(1, 1, 1))  
model_arima_fit = model_arima.fit()


<ipython-input-138-9dd5b5b2b123>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['count'] = pd.to_numeric(train['count'], errors='coerce')


In [139]:

# Fit Prophet model 
train_pr = train.rename(columns={'date': 'ds', 'count': 'y'})

test_x = test.drop(columns=['count'])

test_pr = test_x.rename(columns={'date': 'ds'})

model_prophet = Prophet()

model_prophet.fit(train_pr)

# Fit LSTM model
train_seq = train['count'].values
test_seq = test['date'].values
n_input = 1   # Number of time points to use for prediction

model_lstm = Sequential()
model_lstm.add(LSTM(50, activation='relu', input_shape=(n_input, 1)))
model_lstm.add(Dense(1))
model_lstm.compile(optimizer='adam', loss='mse')
model_lstm.fit(train_seq[:-n_input].reshape(-1,n_input,1), train_seq[n_input:], epochs=100, verbose=0)

# Make predictions
preds_arima = model_arima_fit.forecast(test.shape[0])[:test.shape[0]] 
preds_prophet = model_prophet.predict(test_pr)['yhat'][:test.shape[0]].values



INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpmps_m5p8/suoxezwr.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpmps_m5p8/8r0ftko1.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=10722', 'data', 'file=/tmp/tmpmps_m5p8/suoxezwr.json', 'init=/tmp/tmpmps_m5p8/8r0ftko1.json', 'output', 'file=/tmp/tmpmps_m5p8/prophet_modelz6kewqls/prophet_model-20230601092246.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
09:22:46 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
09:22:46 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


In [140]:
test_x.head()

,date
355,12/22/1988
356,12/23/1988
357,12/24/1988
358,12/25/1988
359,12/26/1988


In [142]:
test_x_lstm = test.drop(['date'], axis=1).values
# preds_lstm = model_lstm.predict(test_x_lstm)

# Ensemble 
# TODO add + preds_lstm
preds = (preds_arima + preds_prophet)/2

# Create submission 
submission = pd.DataFrame({'count': preds})
submission['count'] = submission['count'].astype(int)
submission.to_csv('submission.csv', index=False)